In [261]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [61]:
words = open('names.txt','r').read().splitlines()
len(words)

32033

In [62]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [77]:
#Dataset creation
X,Y = [],[]
block_size = 3
for word in words:
  context = [0]*block_size
  # print(word)
  for w in word + '.':
    ix = stoi[w]
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context) + ' ---> ' + w)
    context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [250]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)

  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [276]:
g = torch.Generator().manual_seed(2147483647)
i = 200
z = 10
y = 3*z
C = torch.randn((27,z),generator = g)
w1 = torch.randn((y,i),generator = g)
b1 = torch.randn((i),generator = g)
w2 = torch.randn((i,27),generator = g)
b2 = torch.randn((27),generator = g)
parameters = [C,w1,b1,w2,b2]


In [277]:
sum(p.nelement() for p in parameters)

11897

In [278]:
for p in parameters:
  p.requires_grad = True

In [279]:
lre = torch.linspace(-3,0,1000)
lrs = 10**lre


In [280]:
#Forward Propagation
# lr = []
# lossa = []
for i in range(300000):
  batch = torch.randint(0,Xtr.shape[0],(32,))
  emb = C[Xtr[batch]]
  h = torch.tanh((emb.view(-1,y) @ w1) + b1)
  logits = (h @ w2) + b2
  loss = F.cross_entropy(logits,Ytr[batch])
  # print(loss.item())

#Backward Propagation
  for p in parameters:
    p.grad = None
  loss.backward()

  lri = 0.1 if i < 200000 else 0.01
  for p in parameters:
    if p.grad == None:
      p.data += -lri * 0.0
    else:
      p.data += -lri * p.grad
  # lr.append(lre[i])
  # lossa.append(loss.item())
# print(loss.item())

In [275]:
print(loss.item())

2.2122182846069336


In [281]:
emb = C[Xtr]
h = torch.tanh((emb.view(-1,y) @ w1) + b1)
logits = (h @ w2) + b2
loss = F.cross_entropy(logits,Ytr)
loss

tensor(2.0788, grad_fn=<NllLossBackward0>)

In [282]:
emb = C[Xdev]
h = torch.tanh((emb.view(-1,y) @ w1) + b1)
logits = (h @ w2) + b2
loss = F.cross_entropy(logits,Ydev )
loss

tensor(2.1464, grad_fn=<NllLossBackward0>)

In [290]:
g = torch.Generator().manual_seed(2147483647 + 10)
for _ in range(30):
  out = []
  context = [0] * block_size
  while True:
    emb = C[torch.tensor([context])]
    h = torch.tanh((emb.view(1,-1) @ w1) + b1)
    logits = (h @ w2) + b2
    probs = F.softmax(logits,dim=1)
    ix = torch.multinomial(probs,num_samples = 1,generator = g).item()
    context = context[1:] + [ix]
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(i for i in out))

mora.
kayah.
see.
med.
ryah.
reivangeldrie.
caishadiel.
izelynne.
reigh.
camanaraelyzimrones.
noshubergiagriel.
kinge.
jenslee.
promi.
brence.
ryyah.
faek.
yuma.
mys.
kyla.
hal.
salynn.
ufia.
jolie.
ren.
cry.
kiviaosien.
adir.
horoe.
bhigpeiva.


In [285]:
emb.shape

torch.Size([3, 10])